In [1]:
import os

In [2]:
%pwd

'/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class PreaperModelCallBacks:
    root_dir : Path 
    tensrboard_root_log_dir: Path
    checkpoint_model_filepath: Path
     

In [6]:
from srcTrackVision.utils.common import read_yaml , create_directories

In [18]:
class ConfigurationManager:
    def __init__(
            self , 
            config_filepath = '/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/config/config.yaml',
            params_filepath = '/Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/params.yaml' ):
        

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # Create the root dir 
        create_directories([self.config.artifacts_root])


    def get_preaper_model_callbacks_config(self) -> PreaperModelCallBacks:
        config = self.config.preaper_callbacks
        model_chpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_chpt_dir) ,
            Path(config.tensrboard_root_log_dir)
        ])
        preaper_model_callbacks_config = PreaperModelCallBacks(
            root_dir= Path(config.root_dir),
            tensrboard_root_log_dir = Path(config.tensrboard_root_log_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
             

        )

    
        return preaper_model_callbacks_config

In [20]:
import time 
import tensorflow as tf 
class PreaperCallBacks:
    def __init__ (self  , config = PreaperModelCallBacks):
        self.config = config

    @property
    def _create_tb_callbacks (self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(self.config.tensrboard_root_log_dir  
                                          , f'tb_logs_at_{timestamp}')
        

        return tf.keras.callbacks.TensorBoard(log_dir = tb_running_log_dir)
    
    @property
    def _create_checkpoint_callbacks (self):
        return tf.keras.callbacks.ModelCheckpoint(filepath= self.config.checkpoint_model_filepath, 
                                                   save_best_only= True
        )
    

    def get_tb_checkpoint_callbacks (self):
        return [
            self._create_tb_callbacks,
            self._create_checkpoint_callbacks
        ]
    
    
                                                          

In [21]:
from srcTrackVision import logger
try:
    config = ConfigurationManager()
    preaper_callbacks_config = config.get_preaper_model_callbacks_config()
    prepaper_model_callbacks = PreaperCallBacks(config= preaper_callbacks_config)
    callback_list =  prepaper_model_callbacks.get_tb_checkpoint_callbacks()


except Exception as e:
    logger.exception(e)
    raise e

2023-06-17 15:16:38,356 - trackvisionlogger - INFO - yaml file /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/config/config.yaml loaded successfully
2023-06-17 15:16:38,358 - trackvisionlogger - INFO - yaml file /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/params.yaml loaded successfully
2023-06-17 15:16:38,359 - trackvisionlogger - INFO - Created directory at: /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/artifacts
2023-06-17 15:16:38,360 - trackvisionlogger - INFO - Created directory at: /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/artifacts/checkpoint_dir
2023-06-17 15:16:38,360 - trackvisionlogger - INFO - Created directory at: /Users/sultanalyami/Desktop/TrackVision_Project/TrackVision/artifacts/preaper_callbacks/tensorboard_log_dir
